In [19]:
import docker
from docker.models.containers import Container

In [31]:
client = docker.from_env()

In [ ]:
kali_id = client.containers.list()[0].id
workdir = codebase_path = "/app/bountybench/vllm/codebase"
command = "mkdir -p ~/miniconda3 && wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-aarch64.sh -O ~/miniconda3/miniconda.sh && bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3 && rm ~/miniconda3/miniconda.sh && source ~/miniconda3/bin/activate && conda init --all && conda create -n vllm_env python=3.9 -y && conda activate vllm_env && export SETUPTOOLS_SCM_PRETEND_VERSION=0.6.2 && export VLLM_TARGET_DEVICE=empty && apt-get update && apt-get install -y git build-essential cmake netcat-openbsd && pip install -e . && pip install pytest msgpack-numpy pytest-asyncio xformers==0.0.27.post2"

exec_params = {
            "container": kali_id,
            "cmd": ["/bin/bash", "-c", command],
            "workdir": workdir,
            "stdin": False,
            "stdout": True,
            "stderr": True,
            "tty": False,
}

In [44]:
import threading
import time
import sys

YELLOW = "\033[93m"
RESET = "\033[0m"

def get_stdout_text(process_output: bytes) -> str:
    """
    Decode the process output from bytes to a UTF-8 string.

    Args:
        process_output (bytes): The output to decode.

    Returns:
        str: The decoded string or an error message if decoding fails.
    """
    stdout: str = ""
    if process_output is None:
        return stdout
    try:
        stdout = process_output.decode("utf-8")
        return stdout
    except UnicodeDecodeError as e:
        stdout = (
            "This failed to decode. Remember: the problem is fully solvable using UTF-8 encoding. "
            "Ignore any images (jpeg, png, etc.), videos (mp4, mpeg) etc. "
            f"Error: {str(e)}"
        )


In [45]:
timeout = 120
exec_id = client.api.exec_create(**exec_params)['Id']
output_stream = client.api.exec_start(exec_id, stream=True, demux=True)

stdout_chunks = []
stderr_chunks = []
stop_event = threading.Event()

def stream_output():
    try:
        for stdout, stderr in output_stream:
            if stop_event.is_set():
                break
            if stdout:
                text = get_stdout_text(stdout)
                print(text, end="")
                stdout_chunks.append(text)
            if stderr:
                text = get_stdout_text(stderr)
                print(f"{YELLOW}{text}{RESET}", end="", file=sys.stderr)
                stderr_chunks.append(text)
    except Exception as e:
        print(f"Exec stream closed: {e}")

stream_thread = threading.Thread(target=stream_output, daemon=True)
stream_thread.start()

# Wait for the exec to finish (poll-based workaround)
exec_inspect = client.api.exec_inspect
start_time = time.time()

while True:
    exec_info = exec_inspect(exec_id)
    if not exec_info["Running"]:
        break
    if time.time() - start_time > timeout:
        stop_event.set()
        print(f"Exec command timed out after {timeout} seconds")
        print(f"Timeout after {timeout} seconds")
    time.sleep(2)

stop_event.set()
stream_thread.join(timeout=5)

stdout_text = "".join(stdout_chunks)
stderr_text = "".join(stderr_chunks)

--2025-04-17 02:11:16--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-aarch64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 127093669 (121M) [application/octet-stream]
Saving to: ‘/root/miniconda3/miniconda.sh’

     0K .......... .......... .......... .......... ..........  0% 42.5M 3s
    50K .......... .......... .......... .......... ..........  0% 73.0M 2s
   100K .......... .......... .......... .......... ..........  0% 14.0M 4s
   150K .......... .......... .......... .......... ..........  0% 50.3M 4s
   200K .......... .......... .......... .......... ..........  0% 10.5M 5s
   250K .......... .......... .......... .......... ..........  0%  116M 5s
   300K .......... .......... .......... .......... ..........  0%  104M 4s
   350K .......... .......... ......

PREFIX=/root/miniconda3
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
no change     /root/miniconda3/condabin/conda
no change     /root/miniconda3/bin/conda
no change     /root/miniconda3/bin/conda-env
no change     /root/miniconda3/bin/activate
no change     /root/miniconda3/bin/deactivate
no change     /root/miniconda3/etc/profile.d/conda.sh
no change     /root/miniconda3/etc/fish/conf.d/conda.fish
no change     /root/miniconda3/shell/condabin/Conda.psm1
no change     /root/miniconda3/shell/condabin/conda-hook.ps1
no change     /root/miniconda3/lib/python3.12/site-packages/xontrib/conda.xsh
no change     /root/miniconda3/etc/profile.d/conda.csh
modified      /root/.bashrc
modified      /root/.zshrc
modified      /root/.config/fish/config.fish
modified      /root/.xonshrc
modified      /root/.tcshrc

==> For changes to take effect, close and re-open your current shell. <



==> WARNING: A newer version of conda exists. <==
    current version: 25.1.1
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c defaults conda









ca-certificates-2025 | 129 KB    | #2         |  12% 




ca-certificates-2025 | 129 KB    | ########## | 100% 




ca-certificates-2025 | 129 KB    | ########## | 100% 


setuptools-75.8.0    | 1.6 MB    | ########## | 100% 
openssl-3.0.16       | 5.2 MB    | ###6       |  36% 

python-3.9.21        | 24.6 MB   | 5          |   6% 





wheel-0.45.1         | 114 KB    | #4         |  14% 





wheel-0.45.1         | 114 KB    | ########## | 100% 





wheel-0.45.1         | 114 KB    | ########## | 100% 


setuptools-75.8.0    | 1.6 MB    | ########## | 100% 


setuptools-75.8.0    | 1.6 MB    | ########## | 100% 
python-3.9.21        | 24.6 MB   | #2         |  12% 

pip-25.0             | 2.3 MB    | ########8  |  89% 

pip-25.0             | 2.3 MB    | ########## | 100% 
python-3.9.21        | 24.6 MB   | #9         |  19% 
openssl-3.0.16       | 5.2 MB    | ########## | 100% 

                                                     
                                            

debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 11.0 MB in 1s (8559 kB/s)
(Reading database ... 281358 files and directories currently installed.)
Preparing to unpack .../git_1%3a2.47.2-0.1_arm64.deb ...
Unpacking git (1:2.47.2-0.1) over (1:2.43.0-1+b1) ...
Preparing to unpack .../git-man_1%3a2.47.2-0.1_all.deb ...
Unpacking git-man (1:2.47.2-0.1) over (1:2.43.0-1) ...
Selecting previously unselected package netcat-openbsd.
Preparing to unpack .../netcat-openbsd_1.228-1_arm64.deb ...
Unpacking netcat-openbsd (1.228-1) ...
Setting up netcat-openbsd (1.228-1) ...
update-alternatives: using /bin/nc.openbsd to provide /bin/nc (nc) in auto mode
Setting up git-man (1:2.47.2-0.1) ...
Setting up git (1:2.47.2-0.1) ...
Processing triggers for man-db (2.12.1-2) ...
Obtaining file:///app/bountybench/vllm/codebase
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished w

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 39.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tomli-2.2.1-py3-none-any.whl.metadata (10 kB)
Using cached tomli-2.2.1-py3-none-any.whl (14 kB)
  Created wheel for xformers: filename=xformers-0.0.27.post2-cp39-cp39-linux_aarch64.whl size=594671 sha256=7b46c647789ed43ebd259e364a79f6f1584112ec77bc074a617274c5ecd39247
  Stored in directory: /root/.cache/pip/wheels/69/fc/d2/d948cf7f311935e7f4ea279e4af38542beb7ce81f0ecf477ed
Successfully built xformers


In [51]:
command = "source ~/miniconda3/bin/activate && conda env list && conda activate vllm_env"
exec_params = {
            "container": kali_id,
            "cmd": ["/bin/bash", "-c", command],
            "workdir": workdir,
            "stdin": False,
            "stdout": True,
            "stderr": True,
            "tty": False,
}

timeout = 120
exec_id = client.api.exec_create(**exec_params)['Id']
output_stream = client.api.exec_start(exec_id, stream=True, demux=True)

stdout_chunks = []
stderr_chunks = []
stop_event = threading.Event()

def stream_output():
    try:
        for stdout, stderr in output_stream:
            if stop_event.is_set():
                break
            if stdout:
                text = get_stdout_text(stdout)
                print(text, end="")
                stdout_chunks.append(text)
            if stderr:
                text = get_stdout_text(stderr)
                print(f"{YELLOW}{text}{RESET}", end="", file=sys.stderr)
                stderr_chunks.append(text)
    except Exception as e:
        print(f"Exec stream closed: {e}")

stream_thread = threading.Thread(target=stream_output, daemon=True)
stream_thread.start()

# Wait for the exec to finish (poll-based workaround)
exec_inspect = client.api.exec_inspect
start_time = time.time()

while True:
    exec_info = exec_inspect(exec_id)
    if not exec_info["Running"]:
        break
    if time.time() - start_time > timeout:
        stop_event.set()
        print(f"Exec command timed out after {timeout} seconds")
        print(f"Timeout after {timeout} seconds")
    time.sleep(2)

stop_event.set()
stream_thread.join(timeout=5)

stdout_text = "".join(stdout_chunks)
stderr_text = "".join(stderr_chunks)


# conda environments:
#
base                 * /root/miniconda3
vllm_env               /root/miniconda3/envs/vllm_env



In [54]:
command = "source ~/miniconda3/bin/activate && conda env list && conda activate vllm_env && python --version"
exec_params = {
            "container": kali_id,
            "cmd": ["/bin/bash", "-c", command],
            "workdir": workdir,
            "stdin": False,
            "stdout": True,
            "stderr": True,
            "tty": False,
}

timeout = 120
exec_id = client.api.exec_create(**exec_params)['Id']
output_stream = client.api.exec_start(exec_id, stream=True, demux=True)

stdout_chunks = []
stderr_chunks = []
stop_event = threading.Event()

def stream_output():
    try:
        for stdout, stderr in output_stream:
            if stop_event.is_set():
                break
            if stdout:
                text = get_stdout_text(stdout)
                print(text, end="")
                stdout_chunks.append(text)
            if stderr:
                text = get_stdout_text(stderr)
                print(f"{YELLOW}{text}{RESET}", end="", file=sys.stderr)
                stderr_chunks.append(text)
    except Exception as e:
        print(f"Exec stream closed: {e}")

stream_thread = threading.Thread(target=stream_output, daemon=True)
stream_thread.start()

# Wait for the exec to finish (poll-based workaround)
exec_inspect = client.api.exec_inspect
start_time = time.time()

while True:
    exec_info = exec_inspect(exec_id)
    if not exec_info["Running"]:
        break
    if time.time() - start_time > timeout:
        stop_event.set()
        print(f"Exec command timed out after {timeout} seconds")
        print(f"Timeout after {timeout} seconds")
    time.sleep(2)

stop_event.set()
stream_thread.join(timeout=5)

stdout_text = "".join(stdout_chunks)
stderr_text = "".join(stderr_chunks)


# conda environments:
#
base                 * /root/miniconda3
vllm_env               /root/miniconda3/envs/vllm_env

Python 3.9.21


In [64]:
kali_id = client.containers.list()[0].id
cmd="""
set -e
apt-get update -qq && apt-get install -y --no-install-recommends \
        build-essential curl wget zlib1g-dev libssl-dev libbz2-dev \
        libreadline-dev libsqlite3-dev libffi-dev liblzma-dev
[ -d "$HOME/.pyenv" ] || curl -fsSL https://pyenv.run | bash

export PYENV_ROOT="$HOME/.pyenv"
export PATH="$PYENV_ROOT/bin:$PYENV_ROOT/shims:$PATH"
eval "$(pyenv init -)"

pyenv install -s 3.9.21
pyenv global 3.9.21
python -V
"""
exec_params = {
            "container": kali_id,
            "cmd": ["/bin/bash", "-c", command],
            "workdir": workdir,
            "stdin": False,
            "stdout": True,
            "stderr": True,
            "tty": False,
}

timeout = 120
exec_id = client.api.exec_create(**exec_params)['Id']
output_stream = client.api.exec_start(exec_id, stream=True, demux=True)

stdout_chunks = []
stderr_chunks = []
stop_event = threading.Event()

def stream_output():
    try:
        for stdout, stderr in output_stream:
            if stop_event.is_set():
                break
            if stdout:
                text = get_stdout_text(stdout)
                print(text, end="")
                stdout_chunks.append(text)
            if stderr:
                text = get_stdout_text(stderr)
                print(f"{YELLOW}{text}{RESET}", end="", file=sys.stderr)
                stderr_chunks.append(text)
    except Exception as e:
        print(f"Exec stream closed: {e}")

stream_thread = threading.Thread(target=stream_output, daemon=True)
stream_thread.start()

# Wait for the exec to finish (poll-based workaround)
exec_inspect = client.api.exec_inspect
start_time = time.time()

while True:
    exec_info = exec_inspect(exec_id)
    if not exec_info["Running"]:
        break
    if time.time() - start_time > timeout:
        stop_event.set()
        print(f"Exec command timed out after {timeout} seconds")
        print(f"Timeout after {timeout} seconds")
    time.sleep(2)

stop_event.set()
stream_thread.join(timeout=5)

stdout_text = "".join(stdout_chunks)
stderr_text = "".join(stderr_chunks)

Hit:1 http://http.kali.org/kali kali-last-snapshot InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.12).
zlib1g-dev is already the newest version (1:1.3.dfsg+really1.3.1-1+b1).
libssl-dev is already the newest version (3.4.0-2).
libbz2-dev is already the newest version (1.0.8-6).
libreadline-dev is already the newest version (8.2-6).
libsqlite3-dev is already the newest version (3.46.1-1).
libffi-dev is already the newest version (3.4.7-1).
liblzma-dev is already the newest version (5.6.3-1+b1).
wget is already the newest version (1.24.5-2).
The following packages were automatically installed and are no longer required:
  finger libavformat60 liblua5.2-0 libnghttp3-3 libpipeline1 libpython3.11-dev
  libssh-gcrypt-4 libwireshark17t64 libwiretap14t64 libwsutil15t64
  libxcb-damage0 libxcb-xv0 libxfont2 man-db python3-git python3-gitdb
  python3-lib2to3 python3-nassl pytho

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   270  100   270    0     0   2064      0 --:--:-- --:--:-- --:--:--  2076
Cloning into '/root/.pyenv'...
Cloning into '/root/.pyenv/plugins/pyenv-doctor'...
Cloning into '/root/.pyenv/plugins/pyenv-update'...
Cloning into '/root/.pyenv/plugins/pyenv-virtualenv'...


# Load pyenv automatically by appending
# the following to 
# ~/.bash_profile if it exists, otherwise ~/.profile (for login shells)
# and ~/.bashrc (for interactive shells) :

export PYENV_ROOT="$HOME/.pyenv"
[[ -d $PYENV_ROOT/bin ]] && export PATH="$PYENV_ROOT/bin:$PATH"
eval "$(pyenv init - bash)"

# Restart your shell for the changes to take effect.

# Load pyenv-virtualenv automatically by adding
# the following to ~/.bashrc:

eval "$(pyenv virtualenv-init -)"

/bin/bash: line 1: pyenv: command not found
